In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

In [2]:
from tqdm import tqdm as notebook_tqdm

In [3]:
import pathlib
import textwrap
from IPython.display import display, Markdown

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text,'> ',predicate=lambda _: True))

In [4]:
loader = PyPDFDirectoryLoader("./Data")
docs = loader.load()

In [5]:
docs

[Document(page_content='1  \n', metadata={'source': 'Data\\20120817022636182.pdf', 'page': 0}),
 Document(page_content='目录 Content \n1  \n目 录 \n \n□卷首语 \n编首语  ................................ ................................ ................................ ..... 2 \n \n \n□他山之玉  \n台湾工业技术研究院运行机制及产学研合作态势  ...............................  3 \n \n关于大学工业研究技术研究院架构和运 行机制的设想  .....................  11 \n \n科技园管理模式的比较与分析  ................................ ............................  25 \n \n美国科技公司工业研究院的发展历程及其决定因素  .........................  29 \n \n \n□经典译文  \n产学研发联合实验室的合作机制分析和案例研究  .............................  32 \n \n服务和创新系统：欧洲技术中心模式  ................................ ................  37 \n \n \n \n \n \n \n ', metadata={'source': 'Data\\20120817022636182.pdf', 'page': 1}),
 Document(page_content='他山之玉  Advice \n2  \n \n \n编首语  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n由于我国自主创新能力薄弱，企业拥有的技术研发能力不足，因此，\n产学研越来越受到社会和企业的

In [6]:
# Text Splitting - Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50)
chunks = text_splitter.split_documents(docs)
len(chunks)

177

In [7]:
chunks[10]

Document(page_content='1 浙江大学公共管理学院 2010年优秀硕士论文，下篇同，鲁良明推荐。  中国台湾工业技术研究院对新竹科技园区和台湾产业技术的发展发挥了要作用。有人\n形象地把工研院与创投业、新竹科技工业园区并列为台湾科技创新体系的“三驾马车” 。\n本文介绍台湾工业技术研究院的性质、组织结构、运作机制。从中可以了解非营利科研机\n构在地区创新系统所处的地位与作用，以及构建与管理非营利科研机构方面的经验。', metadata={'source': 'Data\\20120817022636182.pdf', 'page': 3})

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# 设置本地模型和分词器的路径
local_model_path = './bge-base-en-v1.5'
local_tokenizer_path = './bge-base-en-v1.5'

# 从本地文件加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(local_tokenizer_path)
model = AutoModel.from_pretrained(local_model_path)

# 准备一些文本
text = "Here is some text to encode"

# 使用分词器处理文本
inputs = tokenizer(text, return_tensors="pt")

# 生成嵌入向量
with torch.no_grad():
    outputs = model(**inputs)

# 获取最后一层的隐藏状态作为嵌入向量
embeddings = outputs.last_hidden_state

print(embeddings)


In [8]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

In [9]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [10]:
query ="什么是主题式产学合作?"
search = vectorstore.similarity_search(query)

In [11]:
to_markdown(search[0].page_content)

> 他山之玉  Advice 
> 10 
>  
> 3.主题式产学合作  
> 工研院产学合作最重要的方式是主题式产学合作， 通过与其他研发机构建立
> 互相合作的产学研联合研发中心的方式来促进特定主题的产业研 发， 通过将研发
> 机构的资源和研发能力相结合，并激发研发人员的创新思维，在一定的主题目标
> 下形成良好效果。  
> 产学研联合研发中心的设立是由院长室根据研发主题需要， 指派负责人与合
> 适的合作机构进行洽谈；亦可由单位先行评估后，向院长室提出合作规划（包括
> 合作主题、合作机构、负责洽谈人选等） ，经院长室同意后由负责人整合院内资
> 源并与合作方洽谈。合作的对象应是在该合作主题具有领先地位的学术研究机

In [12]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)

In [13]:
retriever.get_relevant_documents(query)

[Document(page_content='他山之玉  Advice \n10 \n \n3.主题式产学合作  \n工研院产学合作最重要的方式是主题式产学合作， 通过与其他研发机构建立\n互相合作的产学研联合研发中心的方式来促进特定主题的产业研 发， 通过将研发\n机构的资源和研发能力相结合，并激发研发人员的创新思维，在一定的主题目标\n下形成良好效果。  \n产学研联合研发中心的设立是由院长室根据研发主题需要， 指派负责人与合\n适的合作机构进行洽谈；亦可由单位先行评估后，向院长室提出合作规划（包括\n合作主题、合作机构、负责洽谈人选等） ，经院长室同意后由负责人整合院内资\n源并与合作方洽谈。合作的对象应是在该合作主题具有领先地位的学术研究机', metadata={'page': 10, 'source': 'Data\\20120817022636182.pdf'}),
 Document(page_content='主要构建一个以高校为支撑的研究平台 ，称为大学工业技术研究院，简称大学工\n研院。通过多种产学研模式的整合利用和对科研机构的访谈，大学工研院的基本\n创新管理架构如下图所示 ： \n \n \n \n \n \n \n \n \n \n \n \n \n大学工业技术研究院定位于面向应用性的研发机构，通过“政府推动、高校主导、企业支\n撑、国际合作、市场化运作”的运作模式，将高校、企业、政府、科研机构 整合在一个系统之\n中，并对每个主体进行明确的分工和定位。政府为产学研创造环境并提供政策支持；高校和科\n研机构提供技术和智力支持，是系统中的创新源和人才培养基地；企业是研发投入和产业化的', metadata={'page': 11, 'source': 'Data\\20120817022636182.pdf'}),
 Document(page_content='二、合作动力：  \n \n \n 合 \n 作 \n \n \n \n 合作  \n \n \n \n \n \n \n图1：产学研发联合实验室的动力模式  \n本研究发现主要的动力来自于增加解决科技问题的能力和分担风险和成本。  \n三、影响因素：  \n图2：产学研发合作绩效的影响因素模式  \n \n \n \n \n \n 基于同意  \n \n \n \n \n \n

In [17]:
llm = LlamaCpp(
    model_path= "./BioMistral-7B.Q4_K_M.gguf",
    temperature=0.3,
    max_tokens=2048,
    top_p=1)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32           

In [18]:
# RAG Chain
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [19]:
template = """
<|context|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query}
</s>
 <|assistant|>
"""

In [20]:
prompt = ChatPromptTemplate.from_template(template)

In [21]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
response = rag_chain.invoke("什么是技术链?")


llama_print_timings:        load time =     696.30 ms
llama_print_timings:      sample time =      12.06 ms /    88 runs   (    0.14 ms per token,  7295.03 tokens per second)
llama_print_timings: prompt eval time =    3360.84 ms /    56 tokens (   60.02 ms per token,    16.66 tokens per second)
llama_print_timings:        eval time =    8102.41 ms /    88 runs   (   92.07 ms per token,    10.86 tokens per second)
llama_print_timings:       total time =   11657.11 ms /   144 tokens


In [23]:
to_markdown(response)

>  技术链是指连接不同系统或设备的网络。在人工智能领域，通常指连接不同系统或设备的网络，以便于数据交换和通信。在人工智能领域，通常指连接不同系统或设备的网络，以便于数据交换和通信。

In [24]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = rag_chain.invoke(user_input)
  print("Answer: ",result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     696.30 ms
llama_print_timings:      sample time =      16.94 ms /   126 runs   (    0.13 ms per token,  7438.89 tokens per second)
llama_print_timings: prompt eval time =     582.51 ms /    10 tokens (   58.25 ms per token,    17.17 tokens per second)
llama_print_timings:        eval time =   11759.79 ms /   125 runs   (   94.08 ms per token,    10.63 tokens per second)
llama_print_timings:       total time =   12611.05 ms /   135 tokens


Answer:   技术链是指从数据到模型的一系列工作，包括数据收集、预处理、模型选择、模型训练、模型部署等。其中，模型选择和模型训练属于机器学习领域，模型部署属于机器学习和软件工程领域。本次交互主要围绕模型部署进行了交互，因此本次交互主要围绕模型部署进行了交互。
Exiting


SystemExit: 

c:\Users\DELL\miniconda3\envs\langchain\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
